## LESSON 2 - Sebastian Héctor Zuzunaga Rosado - 211006957

In [ ]:
!pip install -Uqq fastai
from fastai.vision.widgets import *

##  Propósito 

## Passo 1: Pesquisar imagens


Primeiro vai ser instalado Duck Duck Go para ser capaz de realizar a busca.

In [ ]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    !pip install -Uqq fastai duckduckgo_search

Os recursos necessários serão importadas e a função para procurar e armazenar as imagens será definida.

In [ ]:
from duckduckgo_search import ddg_images
from fastcore.all import *

def search_images(term, max_images=50):
    print(f"Searching for '{term}'")
    return L(ddg_images(term, max_results=max_images)).itemgot('image')

Serão procuradas imagens de vários tipos de pássaros cantores, divididos em machos e fêmeas devido às diferenças de plumagem encontradas.


In [ ]:
searches = 'wrens','finches','orioles', 'thrushes', 'chickadees', 'cardinal bird'
path = Path('SongBirdOS')
from time import sleep
from fastdownload import download_url
from fastai.vision.all import *

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} male photo'))
    sleep(10)
    download_images(dest, urls=search_images(f'{o} female photo'))
    resize_images(path/o, max_size=100, dest=path/o)

## Passo 2: Treinar o modelo

Se as imagens não são baixadas corretamente isso pode afetar o modelo então eles vão ser excluídos.

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

Para o treinamento será utilizado um datablock o qual se encarregará de criar os dataset de treinamento e verificação, usados para criar o modelo e para a precisão do mesmo, além disso aqui também se configuram os parâmetros essenciais para seu correto funcionamento.

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

dls.show_batch(max_n=6)

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

## Passo 3: Usar o modelo

In [ ]:
urls = search_images('fairy wren', max_images=1) 
urls[0]

In [ ]:
dest = 'Fwren.jpg'
download_url(urls[0], dest, show_progress=False)

im = Image.open(dest)
im.to_thumb(256,256)

In [ ]:
SB_ID,_,probs = learn.predict(PILImage.create('Fwren.jpg'))
print(f"This is a: {SB_ID}.")

In [ ]:
urls = search_images('male northern cardinal', max_images=1)
urls[0]

In [ ]:
dest = 'Ncardinal.jpg'
download_url(urls[0], dest, show_progress=False)

im = Image.open(dest)
im.to_thumb(256,256)

In [ ]:
SB_ID,_,probs = learn.predict(PILImage.create('Ncardinal.jpg'))
print(f"This is a: {SB_ID}.")

In [ ]:
urls = search_images('blue finch', max_images=1)
urls[0]

In [ ]:
dest = 'Bfinch.jpg'
download_url(urls[0], dest, show_progress=False)

im = Image.open(dest)
im.to_thumb(256,256)

In [ ]:
SB_ID,_,probs = learn.predict(PILImage.create('Bfinch.jpg'))
print(f"This is a: {SB_ID}.")

## Data augmentation

Aqui vai criar a matriz de confusão, que nos permitirá ver quantas imagens de cada grupo foram classificadas incorretamente.

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

Estas são as maiores perdas do modelo, estas são dadas porque o modelo está confiante de uma predição incorreta ou se está pouco confiante de uma predição correta.

In [ ]:
interp.plot_top_losses(5, nrows = 2, figsize=(17,4))

O limpador será criado, o que nos permitirá manualmente reclassificar certas imagens ou elimina-las se necessário.

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
for idx in cleaner.delete(): cleaner.fns[idx].unlink()
for idx,cat in cleaner.change(): shutil.mov(str(cleaner.fns[idx]), path/cat)

## Export

Exporta-se o modelo, aqui está tudo o que se aprendeu, isto se faz para poder fazer previsões fora do notebook.

In [ ]:
learn.export('modelsong.pkl')

## Conclusão

Embora o modelo funcione bastante bem, houve um aumento de imprecisões devido ao maior número de categorias, o uso de data augmentation ajuda a diminuir este problema. Conseguir exportar o modelo nos permite dar utilidade às previsões feitas pela inteligência artificial já que não se limita seu uso só ao notebook.

## Aplicação

https://huggingface.co/spaces/sebazac332/minima